# 第３章

3.1 特徴点（キーポイント）

特徴点（キーポイント）とは、点群の中でも特徴的なパターンを持ち、他の点との区別が付きやすいランドマークになりうる点のことを指します。例えば、第4章で紹介する点群レジストレーション（位置合わせ）を考えましょう。点群レジストレーションを行うためには、複数個の異なる視点から撮影した複数の点群データに対して、ある点群データから別の点群データへの座標変換を求めることになります。このとき、点群データ間でどの点とどの点が対応するかを求める（対応点探索をする）必要があります。ここで、一方の点群データ内のすべての点に対する対応点をもう一方の点群データから探すことは計算量の観点から現実的ではありません。さらに、多くの特徴的でない点に対する誤対応点の存在によって点群レジストレーションの精度が下がるかもしれません。以上の理由から、点群レジストレーションの最初の段階で特徴点の抽出が行われます。

本節では2つの特徴点抽出手法を紹介します。まず、Harris3D について述べます。これは2次元画像処理でよく使われる Harris コーナー検出[24]の拡張手法です。次に、Intrinsic Shape Signature (ISS) [84]を紹介します。これはOpen3D をはじめとする多くのオープンソースライブラリに実装されていることから、3次元点群処理でもよく使われる特徴点抽出手法です。

3.1.1 Harris3D

Harris3D は2次元画像処理でよく使われる Harris コーナー検出[24]の拡張手法です。まず、Harris コーナー検出について簡単に説明します。コーナーとなる点は、その点から全方向に対する画素値の変化が大きい点となります。とある窓関数 w(u, v) と画素値 I(u, v) を用いて、とあるピクセルシフト量 (x, y) に対する画素値変化量 E(x, y) は次式で表せます。

```
E(x, y) = Σ w(u, v)|I(x + u, y + v) - I(u, v)|²   (3.1)
```

ただし、全方向を考えるので、(x, y) の集合は {(1, 0), (1, 1), (0, 1), (-1, 1)} とします。窓関数 w(u, v) は局所領域を定義するためのものです。ある大きさの矩形領域でもよいし、滑らかな関数の窓を用いるためのガウス関数 w(u, v) = exp - (u² + v²)/2σ² とすることもあります。ここで、x, y 方向の画素の勾配をそれぞれ Ix, Iy とおき、ピクセルシフト量 (x, y) が十分に小さい値であるとしてテイラー展開を用いると、E(x, y) は式で近似できます。

```
E(x, y) ≈ [x y]M[x/y], M = Σ w(u,v)[IxIx IxIy; IxIy IyIy]   (3.2)
```

行列 M の2つの固有値（仮に λ₁ と λ₂ とします）と点の画素値との間には下記の関係があります。まず、λ₁ と λ₂ が両方小さい場合、すべての方向へのシフトに対して画素値の変化が小さいことになります。次に、λ₁ と λ₂ のどちらか一方が大きく、もう一方が小さい場合は、ある方向にのみ画素値の変化が大きい点、すなわちエッジ上の点であると考えられます。最後に、λ₁ と λ₂ が両方大きい場合は、すべての方向へのシフトに対して画素値の変化が大きい点、すなわちコーナー点であると考えられます。以上をふまえて、λ₁ と λ₂ が両方大きい点を発見するために、次式の Harris 指標 R を計算します。

```
R = det(M) - k tr(M)² = λ₁λ₂ - k(λ₁ + λ₂)²   (3.3)
```

ただし k は実験的に決定するパラメータです注1。コーナー点は R が正の大きな値に、エッジ上の点は R が負の値に、そして特に特徴のないフラットな点は R が正の小さな値になります。よって、R があるしきい値を超える点を抽出すればコーナー点を検出できます。なお、k を明示的に設定することを避けるために、式 (3.3) の代わりに次式が用いられることもあります。

```
R = det(M)/tr(M)   (3.4)
```

最後に、各点に対してある範囲の近傍点をチェックし、自身の R 値よりも小さい R 値を持つ点を消去します。これは、画像中のほぼ同じ位置に多くのコーナー点が重複検出されることを防ぐために局所最大となる点のみを抽出する処理であり、一般に Non Maximum Suppression (NMS) と呼ばれています。

以上が2次元画像処理における Harris コーナー検出の流れです。3次元点群処理のオープンソースライブラリである Point Cloud Library (PCL) では、これを3次元処理に拡張した HarrisKeypoint3D 注2 という関数が用意されています。ここでは、画素値勾配の代わりに点の法線ベクトルを用いて Harris 指標を計算しています。Python のサンプルコードを下記に示します。ただし、式 (3.3) の代わりに式 (3.4) を用いています。